# First steps

This is a minimum working example of what you can already with GMT/Python.

In [ ]:
import gmt

In [ ]:
# Start a new figure. 
# Optional if this is the only figure (but recommended).
gmt.figure()
# Creat a global basemap with the Mollweide projection
gmt.psbasemap(R='g', J='W7i', B='afg', P=True)
# Unlike the GMT command-line interface, no figure file is generated unless
# savefig or psconvert are called.
gmt.psconvert(F='myfigure', T='g', A=True)

In [ ]:
# Display the figure in the notebook
from IPython.display import Image, display
Image(filename='myfigure.png', embed=True, width=700)

In [ ]:
!rm myfigure.png